## Setup


*   Perform imports and installs
*   Retrieve OpenAI API key
*   Define JSON format for output of translation text completion



In [2]:
!pip install openai pyairtable

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 99.9/99.9 kB 7.3 MB/s eta 0:00:00


In [3]:
import openai, json
from openai import OpenAI
from pydantic import BaseModel

from google.colab import userdata
openai.api_key = userdata.get("OPENAI_API_KEY")

class output(BaseModel):
  text: str

## Take user input


*   Text to translate and output language is hardcoded



In [9]:
inputtext = """엄마 걱정
기형도
열무 삼십 단을 이고
시장에 간 우리 엄마
안 오시네, 해는 시든 지 오래
나는 찬밥처럼 방에 담겨
아무리 천천히 숙제를 해도
엄마 안 오시네, 배춧잎 같은 발소리 타박타박
안 들리네, 어둡고 무서워
금간 창 틈으로 고요히 빗소리
빈 방에 혼자 엎드려 훌쩍거리던
아주 먼 옛날
지금도 내 눈시울을 뜨겁게 하는
그 시절, 내 유년의 윗목"""

inputlanguage = "Korean"
outputlanguage = "English"

system_prompt = "You are a native speaker, who pays attention to translational nuance."

## Translation via text completion
* Perform text completion
* Print results

In [10]:
client = OpenAI(api_key=openai.api_key)

def translatebasic(outputlang, inputtext, systemprompt):
    response = client.responses.create(
        model="gpt-4.1",
        input=f"Translate the following text to {outputlang}, keeping in mind the following instructions: {systemprompt}. Text: {inputtext}. ONLY RETURN THE TRANSLATION. ONLY THE TEXT-- do not thank me or ask questions- only text.",
    )
    final = response.output_text
    return(final)

translated_text = translatebasic(outputlanguage, inputtext, system_prompt)
print(translated_text)

Mother, My Worry  
Gi Hyeong-do

Carrying thirty bundles of young radish  
My mother went to the market  
She’s not home yet, the sun has long since faded  
I sit in my room, set aside like cold rice  
No matter how slowly I do my homework  
Mother doesn’t return; her cabbage-leaf footsteps  
Don’t echo down the hall, thump thump  
I can’t hear them, it’s dark and frightening  
Through the cracked window, the quiet sound of rain  
Long ago, alone in an empty room,  
I lay facedown, stifling my sobs  
Those distant days still bring tears to my eyes  
Those days, the upper corner of my childhood room.


In [17]:
num_rounds = 3


def opposite_prompt(text: str, lang: str) -> str:
    """
    Return the semantic opposite of `text` (still in `lang`).
    """
    resp = client.responses.create(
        model="gpt-4o",
        input=[
            {"role": "system",
             "content": "Provide the semantic opposite of the user's text. "
                        "Return ONLY the opposite—no commentary."},
            {"role": "user", "content": text},
        ],
    )
    return resp.output_text.strip()

step_text = translatebasic(
    outputlanguage,
    inputtext,
    "You can speak every language as a native perfectly."
)
print("ROUND 0  (inputlanguage → outputlanguage)\n")
print(step_text, "\n")

for r in range(1, num_rounds + 1):
    back_to_input = translatebasic(
        inputlanguage,
        step_text,
        "Translate accurately, no extra words."
    )

    opposite_in_input = opposite_prompt(back_to_input, inputlanguage)
    print(f"ROUND {r}a  ➜  opposite in {inputlanguage}\n")
    print(opposite_in_input, "\n")

    step_text = translatebasic(
        outputlanguage,
        opposite_in_input,
        "Translate accurately, no extra words."
    )
    print(f"ROUND {r}b  ➜  back-translated to {outputlanguage}\n")
    print(step_text, "\n")

ROUND 0  (inputlanguage → outputlanguage)

Worrying About Mom  
Ki Hyung-do  

With thirty bunches of young radish on her head  
My mother went to the market  
She does not return, the sun has withered for ages  
I am tucked away in my room like cold rice  
No matter how slowly I do my homework  
Mother does not return, her footsteps, like cabbage leaves,  
Pat-pat, cannot be heard, it’s dark and scary  
Through the cracked window quietly, sound of rain  
Alone in the empty room, lying face down, I used to sob  
A very long time ago  
That time, which still warms my eyes with tears  
The upper corner of my childhood days. 

ROUND 1a  ➜  opposite in Korean

아빠 안심  
형형상  
  
희미한 천이 가볍게 내려앉아  
아빠는 집에 계셨다  
햇빛이 막 피어오르자 아빠는 여전히 곁에 있다  
나는 따뜻한 밥 같은 집안에 있었다  
아무리 빨리 공부를 끝내도  
아빠는 여전히 계셨다, 선명한 발소리  
쿵쾅, 잘 들린다, 밝고 안전하다  
온전한 창문으로 활기차게 들려오는 새소리  
가득 찬 방에서 기뻐 웃곤 했다  
별로 멀지 않은  
여전히 웃음으로 눈을 빛내던 그 시절  
내 어린 시절의 낮은 모퉁이 

ROUND 1b  ➜  back-translated to English

Dad’s Reassurance  
Hyun Hyung-sang  
